# Classes and Functions


In [20]:
import requests
import pandas as pd


class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key

    def fetch_intraday_data(self, symbol):
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={self.api_key}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()["Time Series (5min)"]
        except requests.exceptions.HTTPError as err:
            print(f"Error occurred while fetching data for {script}: {err}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching intraday data for {symbol}: {e}")
            return None

    def convert_intraday_data(self, symbol):
        data = self.fetch_intraday_data(symbol)
        if data is None:
            return None

        df = pd.DataFrame(columns=["timestamp", "open", "high", "low", "close", "volume"])
        for timestamp, values in data.items(): # Refer to README.md for explanation of this loop
            row = {
                "timestamp": pd.Timestamp(timestamp),
                "open": float(values["1. open"]),
                "high": float(values["2. high"]),
                "low": float(values["3. low"]),
                "close": float(values["4. close"]),
                "volume": int(values["5. volume"])
            }
            df = pd.concat([df, pd.DataFrame(row, index=[0])], ignore_index=True) # since append method is being deprecated
        return df

    def __getitem__(self, key):
        return self.convert_intraday_data(key)

    def __setitem__(self, key, value):
        raise NotImplementedError("ScriptData does not support item assignment")

    def __contains__(self, key):
        return self.convert_intraday_data(key) is not None

    
def indicator1(df, timeperiod):
    # Calculate the moving average of the 'close' column
    ma = df['close'].rolling(window=timeperiod).mean()
    
    # Create a new DataFrame with the 'timestamp' and 'indicator' columns
    new_df = pd.DataFrame({'timestamp': df['timestamp'], 'indicator': ma})
    
    
    return new_df


class Strategy:
    def __init__(self, api_key):
        self.sd = ScriptData(api_key)
        self.close_data = None
        self.indicator_data = None

    def fetch_data(self, script):
        self.df = self.sd.convert_intraday_data(script)
        self.close_data = self.df['close']
        self.indicator_data = indicator1(self.df, 5)['indicator']

    def generate_signals(self):
        if self.close_data is None or self.indicator_data is None:
            print('Data not fetched yet. Please call fetch_data() first.')
            return

        try:
            signals = pd.DataFrame({
                'timestamp': self.df['timestamp'],
                'signal': 'NO_SIGNAL'
            })
            for i in range(1, len(self.close_data)): #Refer to README.md for explanation of this loop
                if self.indicator_data[i] > self.close_data[i] and self.indicator_data[i - 1] <= self.close_data[i - 1]:
                    signals.loc[i, 'signal'] = 'BUY'
                elif self.indicator_data[i] < self.close_data[i] and self.indicator_data[i - 1] >= self.close_data[i - 1]:
                    signals.loc[i, 'signal'] = 'SELL'
        except Exception as e:
            print('Error generating signals:', e)
            return

        signals = signals.loc[signals['signal'].isin(['BUY', 'SELL'])].reset_index(drop=True) #filter out NO_SIGNAL
        print(signals)



# Running the code with examples

## Script Data Class

In [21]:
script_data = ScriptData('LJDOMOY6XH5AF0QP')
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,97.120,97.1200,97.1000,97.1000,5305
1,2023-02-15 19:55:00,97.060,97.1200,97.0600,97.1200,11828
2,2023-02-15 19:50:00,97.000,97.0500,97.0000,97.0500,6288
3,2023-02-15 19:45:00,96.950,97.0000,96.9100,96.9800,18206
4,2023-02-15 19:40:00,96.910,96.9500,96.9100,96.9500,3683
...,...,...,...,...,...,...
95,2023-02-15 12:05:00,96.230,96.4450,96.2201,96.4301,710822
96,2023-02-15 12:00:00,96.460,96.5500,96.2100,96.2236,572605
97,2023-02-15 11:55:00,96.595,96.6100,96.3600,96.4507,477423
98,2023-02-15 11:50:00,96.800,96.8468,96.3700,96.5900,1078605


In [22]:
'GOOGL' in script_data

True

## Indicator1 class

In [23]:
indicator1(script_data['GOOGL'],5)

,timestamp,indicator
0,2023-02-15 20:00:00,NaN
1,2023-02-15 19:55:00,NaN
2,2023-02-15 19:50:00,NaN
3,2023-02-15 19:45:00,NaN
4,2023-02-15 19:40:00,97.04000
...,...,...
95,2023-02-15 12:05:00,96.21702
96,2023-02-15 12:00:00,96.26974
97,2023-02-15 11:55:00,96.29988
98,2023-02-15 11:50:00,96.35488


## Strategy Class

In [24]:
strategy = Strategy('LJDOMOY6XH5AF0QP')
strategy.fetch_data('NVDA')
strategy.generate_signals()

             timestamp signal
0  2023-02-15 19:30:00   SELL
1  2023-02-15 19:10:00    BUY
2  2023-02-15 18:40:00   SELL
3  2023-02-15 18:30:00    BUY
4  2023-02-15 18:10:00   SELL
5  2023-02-15 18:05:00    BUY
6  2023-02-15 18:00:00   SELL
7  2023-02-15 17:30:00    BUY
8  2023-02-15 17:05:00   SELL
9  2023-02-15 16:55:00    BUY
10 2023-02-15 16:50:00   SELL
11 2023-02-15 16:35:00    BUY
12 2023-02-15 16:30:00   SELL
13 2023-02-15 16:05:00    BUY
14 2023-02-15 15:15:00   SELL
15 2023-02-15 14:35:00    BUY
16 2023-02-15 14:15:00   SELL
17 2023-02-15 14:05:00    BUY
18 2023-02-15 13:25:00   SELL
19 2023-02-15 13:15:00    BUY
20 2023-02-15 13:00:00   SELL
21 2023-02-15 12:40:00    BUY
22 2023-02-15 12:20:00   SELL
23 2023-02-15 12:10:00    BUY
24 2023-02-15 12:00:00   SELL
25 2023-02-15 11:45:00    BUY
